In [ ]:
from collections import namedtuple
import matplotlib.pyplot as plt
import torch
from peakweather.dataset import PeakWeatherDataset
from torch.utils.data import Dataset, DataLoader
from copy import copy

In [ ]:
class PWDataset(Dataset):

    Sample = namedtuple("Sample", ["x", "y", "mu", "sigma"])

    def __init__(self, window: int, horizon: int, parameter: str = "temperature"):
        ds = PeakWeatherDataset(
            root=None,
            compute_uv=False,
            station_type="meteo_station",
            freq="h",
            aggregation_methods={'temperature': 'mean'},
        )
        self.mode = "train"
        self.window = window
        self.horizon = horizon
        train, mask = ds.get_observations(parameters=parameter, first_date="2020-01-01", last_date="2020-11-30", as_numpy=True, return_mask=True)
        good_stations = (mask.sum(axis=0) > 0).squeeze()
        self.data = {
            "train": train[:, good_stations].squeeze(),
            "val": ds.get_observations(parameters=parameter, first_date="2020-12-01", last_date="2020-12-31", as_numpy=True)[:, good_stations].squeeze(),
            "test": ds.get_observations(parameters=parameter, first_date="2021-01-01", last_date="2021-01-31", as_numpy=True)[:, good_stations].squeeze(),
        }
        self.scaling_params = {"mu": self.data["train"].mean(axis=0), "sigma": self.data["train"].std(axis=0)}
        for mode in ["train", "val", "test"]:
            self.data[mode] = (self.data[mode] - self.scaling_params["mu"]) / self.scaling_params["sigma"]

    def __len__(self):
        return (self.data[self.mode].shape[0] - self.window - self.horizon) * self.data[self.mode].shape[1]

    def __getitem__(self, idx):
        n, t = divmod(idx, (self.data[self.mode].shape[0] - self.window - self.horizon))
        x = self.data[self.mode][t:t + self.window, n]
        y = self.data[self.mode][t + self.window:t + self.window + self.horizon, n]
        mu = self.scaling_params["mu"][n]
        sigma = self.scaling_params["sigma"][n]
        return self.Sample(
            x=torch.tensor(x, dtype=torch.float32),
            y=torch.tensor(y, dtype=torch.float32),
            mu=torch.tensor(mu, dtype=torch.float32),
            sigma=torch.tensor(sigma, dtype=torch.float32),
        )

class GRUModel(torch.nn.Module):
    def __init__(self, horizon: int, hidden_size: int = 16, num_layers: int = 2, dropout: float = 0.0):
        super().__init__()
        self.gru = torch.nn.GRU(input_size=1, hidden_size=hidden_size, batch_first=True, num_layers=num_layers, dropout=dropout)
        self.fc = torch.nn.Linear(in_features=hidden_size, out_features=horizon)

    def forward(self, x: torch.Tensor):
        x = x.unsqueeze(-1)  # add feature dim
        _, h = self.gru(x)
        out = self.fc(h[-1].squeeze(0))
        return out

class MLPModel(torch.nn.Module):
    def __init__(self, window: int, horizon: int, hidden_size: int = 16, num_layers: int = 2, dropout: float = 0.0):
        super().__init__()
        layers = []
        for i in range(num_layers):
            layers.append(torch.nn.Linear(in_features=window if i == 0 else hidden_size, out_features=hidden_size))
            layers.append(torch.nn.ReLU())
            if dropout > 0.0:
                layers.append(torch.nn.Dropout(p=dropout))
        self.mlp = torch.nn.Sequential(*layers)
        self.fc = torch.nn.Linear(in_features=hidden_size, out_features=horizon)

    def forward(self, x: torch.Tensor):
        h = self.mlp(x)
        out = self.fc(h)
        return out

In [ ]:
def train_model(model: torch.nn.Module, lr: float, epochs: int, train_loader: DataLoader, val_loader: DataLoader) ->torch.nn.Module:
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    best_val_loss = float("inf")
    best_state = None

    for epoch in range(epochs):
        model.train()
        total_train = 0
        for batch in train_loader:
            optimizer.zero_grad()
            preds = model(batch.x)
            loss = criterion(preds, batch.y)
            loss.backward()
            optimizer.step()
            total_train += loss.item()
        avg_train = total_train / len(train_loader)

        model.eval()
        total_val = 0
        with torch.no_grad():
            for batch in val_loader:
                preds = model(batch.x)
                loss = criterion(preds, batch.y)
                total_val += loss.item()
        avg_val = total_val / len(val_loader)

        # Save best model
        if avg_val < best_val_loss:
            best_val_loss = avg_val
            best_state = model.state_dict()

        print(f"Epoch {epoch+1}/{epochs} | Train: {avg_train:.4f} | Val: {avg_val:.4f}")

    # Restore best parameters
    model.load_state_dict(best_state)
    print(f"✅ Restored best model (val loss = {best_val_loss:.4f})")
    return model

def test_model(model: torch.nn.Module, test_loader: DataLoader) ->float:
    criterion = torch.nn.L1Loss()
    model.eval()
    total_test = 0
    with torch.no_grad():
        for batch in test_loader:
            preds = model(batch.x)
            preds_rescaled = preds * batch.sigma.unsqueeze(-1) + batch.mu.unsqueeze(-1)
            y_rescaled = batch.y * batch.sigma.unsqueeze(-1) + batch.mu.unsqueeze(-1)
            loss = criterion(preds_rescaled, y_rescaled)
            total_test += loss.item()
    avg_test = total_test / len(test_loader)
    print(f"Test MAE: {avg_test:.4f}")
    return avg_test

def plot_predictions(model: torch.nn.Module, test_dataset: Dataset, num_samples: int = 5):
    model.eval()
    fig, axs = plt.subplots(num_samples, 1, figsize=(10, num_samples * 3))
    with torch.no_grad():
        for i in range(num_samples):
            n = torch.randint(0, len(test_dataset), (1,)).item()
            sample = test_dataset[n]
            preds = model(sample.x.unsqueeze(0)).squeeze(0).numpy()
            # Rescale predictions and ground truth
            preds_rescaled = preds * sample.sigma.numpy() + sample.mu.numpy()
            y_rescaled = sample.y.numpy() * sample.sigma.numpy() + sample.mu.numpy()
            axs[i].plot(range(len(sample.x)), sample.x.numpy() * sample.sigma.numpy() + sample.mu.numpy(), label="input window")
            axs[i].plot(range(len(sample.x), len(sample.x) + len(y_rescaled)), y_rescaled, label="true horizon")
            axs[i].plot(range(len(sample.x), len(sample.x) + len(preds_rescaled)), preds_rescaled, label="forecast")
            axs[i].legend()
            axs[i].set_title(f"Sample {n}")
    plt.tight_layout()
    plt.show()

In [ ]:
# Select window and horizon length
window, horizon = 96, 24
# Create datasets
train_dataset = PWDataset(window=window, horizon=horizon, parameter="temperature")
val_dataset = copy(train_dataset)
val_dataset.mode = "val"
test_dataset = copy(train_dataset)
test_dataset.mode = "test"

In [ ]:
# Create DataLoaders
batch_size = 8192
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
#model = GRUModel(horizon=horizon, hidden_size=8, num_layers=2, dropout=0.0)
model = MLPModel(window=window, horizon=horizon, hidden_size=16, num_layers=2, dropout=0.0)

In [ ]:
model = train_model(
    model=model,
    lr=0.001,
    epochs=5,
    train_loader=train_dataloader,
    val_loader=val_dataloader,
)

In [ ]:
mae = test_model(model, test_dataloader)

In [ ]:
plot_predictions(model, test_dataset, num_samples=5)